# 🌟 BONUS: Deep Agents with Reflection Patterns

## 🎯 Advanced Agentic AI with Self-Reflection

### What Are Deep Agents?

**Deep Agents** are an advanced agent architecture designed for handling complex, multi-step tasks that require:
- 🧠 **Planning capability**: Break down large tasks into subtasks
- 💭 **Self-reflection**: Critique and improve their own outputs
- 🔄 **Context management**: Retain information across long conversations
- 🤖 **Sub-agent delegation**: Launch specialized agents for focused tasks
- 📁 **File system integration**: Persist and retrieve memory

### Real-World Applications:

- 🔍 **Deep Research**: Multi-step investigation and synthesis
- 💻 **Code Generation**: Generate, test, and refine code
- 📊 **Data Analysis**: Complex analytical workflows
- 📝 **Content Creation**: Draft, review, and improve writing

### Reflection Pattern:

Reflection is a prompting strategy where an LLM:
1. **Generates** an initial response
2. **Critiques** its own work
3. **Revises** based on self-assessment
4. **Repeats** until quality standards are met

```
┌─────────────┐
│   Generate  │
└──────┬──────┘
       │
       ▼
┌─────────────┐
│   Reflect   │ ◄──┐
└──────┬──────┘    │
       │           │
       ▼           │
┌─────────────┐    │
│   Revise    │────┘
└──────┬──────┘
       │
       ▼
    Quality
 Threshold Met?
```

Let's build advanced reflection agents for BakeryAI! 🚀

In [ ]:
!pip install -q langgraph langchain langchain-openai
!pip install -q python-dotenv tavily-python

In [ ]:
import os
from typing import TypedDict, Annotated, List
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

print("✅ Environment ready for Deep Agents!")

## 1. Basic Reflection Pattern

Start with simple generate → reflect → revise cycle.

In [ ]:
class ReflectionState(TypedDict):
    task: str
    generation: str
    reflection: str
    revision_count: int
    max_revisions: int
    final_output: str

def generate_content(state: ReflectionState) -> ReflectionState:
    """Generate initial content"""
    print(f"\n📝 Generating content (attempt {state['revision_count'] + 1})...")
    
    if state['revision_count'] == 0:
        # Initial generation
        prompt = f"""Create a professional response for this task:
        
        Task: {state['task']}
        
        Provide a complete, well-structured response."""
    else:
        # Revision based on reflection
        prompt = f"""Improve this response based on the critique:
        
        Original Task: {state['task']}
        
        Previous Response:
        {state['generation']}
        
        Critique:
        {state['reflection']}
        
        Provide an improved version that addresses the critique."""
    
    response = llm.invoke(prompt)
    state['generation'] = response.content
    
    print(f"   Generated {len(response.content)} characters")
    return state

def reflect_on_content(state: ReflectionState) -> ReflectionState:
    """Reflect on generated content"""
    print("\n🤔 Reflecting on content...")
    
    reflection_prompt = f"""You are a critic. Analyze this response:
    
    Task: {state['task']}
    
    Response:
    {state['generation']}
    
    Provide constructive criticism focusing on:
    1. Completeness: Does it fully address the task?
    2. Clarity: Is it clear and well-structured?
    3. Accuracy: Is the information correct?
    4. Professionalism: Is the tone appropriate?
    
    If the response is excellent, say "APPROVED".
    Otherwise, provide specific improvements needed."""
    
    critique = llm.invoke(reflection_prompt)
    state['reflection'] = critique.content
    
    print(f"   Critique: {critique.content[:100]}...")
    return state

def should_continue(state: ReflectionState) -> str:
    """Decide whether to continue revising"""
    
    # Check if approved
    if "APPROVED" in state['reflection'].upper():
        print("\n✅ Content APPROVED!")
        return "finalize"
    
    # Check revision limit
    state['revision_count'] += 1
    if state['revision_count'] >= state['max_revisions']:
        print(f"\n⏸️  Max revisions ({state['max_revisions']}) reached")
        return "finalize"
    
    print("\n🔄 Revising...")
    return "revise"

def finalize_output(state: ReflectionState) -> ReflectionState:
    """Finalize the output"""
    state['final_output'] = state['generation']
    print("\n🎉 Final output ready!")
    return state

# Build reflection graph
reflection_graph = StateGraph(ReflectionState)

reflection_graph.add_node("generate", generate_content)
reflection_graph.add_node("reflect", reflect_on_content)
reflection_graph.add_node("finalize", finalize_output)

reflection_graph.set_entry_point("generate")

reflection_graph.add_edge("generate", "reflect")

reflection_graph.add_conditional_edges(
    "reflect",
    should_continue,
    {
        "revise": "generate",
        "finalize": "finalize"
    }
)

reflection_graph.add_edge("finalize", END)

reflection_app = reflection_graph.compile()

print("✅ Basic reflection agent created!")

In [ ]:
# Test basic reflection
print("🧪 Testing Basic Reflection Agent")
print("="*70)

task = """Write a customer email responding to a complaint about a delayed cake order. 
The order was delayed due to supply chain issues with specialty ingredients."""

result = reflection_app.invoke({
    "task": task,
    "generation": "",
    "reflection": "",
    "revision_count": 0,
    "max_revisions": 3,
    "final_output": ""
})

print("\n" + "="*70)
print("📧 FINAL EMAIL:")
print("="*70)
print(result['final_output'])
print("\n" + "="*70)
print(f"Revisions made: {result['revision_count']}")

## 2. Reflexion Agent with External Tools

Advanced reflection with tool usage and grounded critique.

In [ ]:
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults

@tool
def check_bakery_policy(policy_type: str) -> str:
    """Check BakeryAI policies
    
    Args:
        policy_type: Type of policy (refund, delivery, allergen, etc.)
    """
    policies = {
        "refund": "Full refund within 24 hours of order. Store credit after 24 hours.",
        "delivery": "Free delivery on orders over $100. $10 fee for orders under $100.",
        "allergen": "All products may contain traces of nuts, dairy, eggs, wheat.",
        "cancellation": "Cancel up to 12 hours before delivery for full refund."
    }
    return policies.get(policy_type.lower(), "Policy not found")

@tool
def check_ingredient_availability(ingredient: str) -> str:
    """Check ingredient availability
    
    Args:
        ingredient: Name of ingredient
    """
    import random
    available = random.random() > 0.3
    if available:
        return f"{ingredient} is currently in stock"
    return f"{ingredient} is temporarily out of stock - expected in 2-3 days"

tools = [check_bakery_policy, check_ingredient_availability]

# Optional: Add web search
try:
    web_search = TavilySearchResults(max_results=2)
    tools.append(web_search)
    print("✅ Web search tool added")
except:
    print("⚠️  Tavily not configured (optional)")

print(f"✅ {len(tools)} tools available for reflexion agent")

In [ ]:
from langchain.agents import create_openai_functions_agent, AgentExecutor

class ReflexionState(TypedDict):
    task: str
    draft: str
    search_queries: List[str]
    tool_results: List[str]
    critique: str
    revision: str
    iterations: int
    max_iterations: int

def draft_response(state: ReflexionState) -> ReflexionState:
    """Generate initial draft with tool usage"""
    print(f"\n📝 Drafting response (iteration {state['iterations'] + 1})...")
    
    if state['iterations'] == 0:
        # Initial draft
        prompt_template = ChatPromptTemplate.from_messages([
            ("system", """You are a helpful BakeryAI assistant.
            Draft a response to the customer's request.
            Use tools to verify policies and check information.
            
            Generate search queries if you need more information."""),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        
        agent = create_openai_functions_agent(llm, tools, prompt_template)
        agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
        
        result = agent_executor.invoke({"input": state['task']})
        state['draft'] = result['output']
    else:
        # Revision based on critique
        revision_prompt = f"""Improve this draft based on the critique:
        
        Task: {state['task']}
        
        Previous Draft:
        {state['draft']}
        
        Critique:
        {state['critique']}
        
        Tool Results:
        {chr(10).join(state['tool_results'])}
        
        Provide improved version addressing all issues."""
        
        response = llm.invoke(revision_prompt)
        state['draft'] = response.content
    
    print(f"   Draft length: {len(state['draft'])} chars")
    return state

def execute_searches(state: ReflexionState) -> ReflexionState:
    """Execute tool calls for verification"""
    print("\n🔍 Executing verification checks...")
    
    # Simulate tool execution
    tool_results = []
    
    # Check relevant policies
    for policy in ["refund", "delivery"]:
        result = check_bakery_policy.invoke({"policy_type": policy})
        tool_results.append(f"{policy.title()} Policy: {result}")
    
    state['tool_results'] = tool_results
    print(f"   Retrieved {len(tool_results)} tool results")
    return state

def critique_draft(state: ReflexionState) -> ReflexionState:
    """Critique draft with external grounding"""
    print("\n🔎 Critiquing draft...")
    
    critique_prompt = f"""You are an expert critic. Analyze this customer service response:
    
    Task: {state['task']}
    
    Draft Response:
    {state['draft']}
    
    Verified Facts:
    {chr(10).join(state['tool_results'])}
    
    Critique the response on:
    1. **Accuracy**: Does it match verified policies?
    2. **Completeness**: Does it address all aspects?
    3. **Tone**: Is it empathetic and professional?
    4. **Citations**: Does it reference policies correctly?
    
    If excellent, say "APPROVED".
    Otherwise, enumerate specific issues to fix."""
    
    critique = llm.invoke(critique_prompt)
    state['critique'] = critique.content
    
    print(f"   Critique: {critique.content[:80]}...")
    return state

def should_revise(state: ReflexionState) -> str:
    """Decide whether to revise"""
    if "APPROVED" in state['critique'].upper():
        print("\n✅ Draft APPROVED!")
        return END
    
    state['iterations'] += 1
    if state['iterations'] >= state['max_iterations']:
        print(f"\n⏸️  Max iterations ({state['max_iterations']}) reached")
        return END
    
    print("\n🔄 Needs revision")
    return "draft"

# Build Reflexion graph
reflexion_graph = StateGraph(ReflexionState)

reflexion_graph.add_node("draft", draft_response)
reflexion_graph.add_node("search", execute_searches)
reflexion_graph.add_node("critique", critique_draft)

reflexion_graph.set_entry_point("draft")

reflexion_graph.add_edge("draft", "search")
reflexion_graph.add_edge("search", "critique")

reflexion_graph.add_conditional_edges(
    "critique",
    should_revise,
    {
        "draft": "draft",
        END: END
    }
)

reflexion_app = reflexion_graph.compile()

print("✅ Reflexion agent with tools created!")

In [ ]:
# Test Reflexion agent
print("🧪 Testing Reflexion Agent with Tools")
print("="*70)

task = """A customer wants to cancel their wedding cake order that's scheduled 
for delivery tomorrow. They're asking about refund options. Write a response."""

result = reflexion_app.invoke({
    "task": task,
    "draft": "",
    "search_queries": [],
    "tool_results": [],
    "critique": "",
    "revision": "",
    "iterations": 0,
    "max_iterations": 3
})

print("\n" + "="*70)
print("📧 FINAL RESPONSE:")
print("="*70)
print(result['draft'])
print("\n" + "="*70)
print("\n📋 VERIFICATION:")
for tool_result in result['tool_results']:
    print(f"  • {tool_result}")
print("\n" + "="*70)
print(f"Iterations: {result['iterations']}")

## 3. Deep Agent with Planning & Sub-Agents

Full deep agent implementation with task decomposition.

In [ ]:
class DeepAgentState(TypedDict):
    objective: str
    plan: List[str]
    current_step: int
    step_results: List[str]
    reflections: List[str]
    final_output: str

def create_plan(state: DeepAgentState) -> DeepAgentState:
    """Decompose objective into steps"""
    print("\n📋 Creating execution plan...")
    
    planning_prompt = f"""Break down this objective into 3-5 concrete steps:
    
    Objective: {state['objective']}
    
    Provide a numbered list of specific, actionable steps.
    Each step should be clear and measurable."""
    
    response = llm.invoke(planning_prompt)
    
    # Parse steps
    steps = []
    for line in response.content.split('\n'):
        line = line.strip()
        if line and (line[0].isdigit() or line.startswith('-') or line.startswith('*')):
            # Remove numbering
            step = line.lstrip('0123456789.-* ')
            if step:
                steps.append(step)
    
    state['plan'] = steps
    state['current_step'] = 0
    
    print("\n📌 Execution Plan:")
    for i, step in enumerate(steps, 1):
        print(f"   {i}. {step}")
    
    return state

def execute_step(state: DeepAgentState) -> DeepAgentState:
    """Execute current step with sub-agent"""
    step_num = state['current_step']
    step = state['plan'][step_num]
    
    print(f"\n⚙️  Executing Step {step_num + 1}/{len(state['plan'])}: {step}")
    
    # Create specialized sub-agent for this step
    execution_prompt = f"""You are a specialized sub-agent for this task:
    
    Overall Objective: {state['objective']}
    Current Step: {step}
    
    Previous Results:
    {chr(10).join([f"Step {i+1}: {r}" for i, r in enumerate(state['step_results'])])}
    
    Execute this step and provide concrete results.
    Be specific and actionable."""
    
    result = llm.invoke(execution_prompt)
    state['step_results'].append(result.content)
    
    print(f"   ✓ Result: {result.content[:80]}...")
    return state

def reflect_on_progress(state: DeepAgentState) -> DeepAgentState:
    """Reflect on step execution"""
    print("\n🤔 Reflecting on progress...")
    
    reflection_prompt = f"""Reflect on the execution of this step:
    
    Step: {state['plan'][state['current_step']]}
    Result: {state['step_results'][-1]}
    
    Assess:
    1. Was the step completed successfully?
    2. Is the result sufficient for proceeding?
    3. Any issues or gaps?
    
    Provide brief reflection (2-3 sentences)."""
    
    reflection = llm.invoke(reflection_prompt)
    state['reflections'].append(reflection.content)
    
    print(f"   Reflection: {reflection.content[:60]}...")
    return state

def should_continue_plan(state: DeepAgentState) -> str:
    """Check if more steps remain"""
    state['current_step'] += 1
    
    if state['current_step'] < len(state['plan']):
        return "execute"
    
    print("\n✅ All steps completed!")
    return "synthesize"

def synthesize_results(state: DeepAgentState) -> DeepAgentState:
    """Synthesize all results into final output"""
    print("\n🎯 Synthesizing final output...")
    
    synthesis_prompt = f"""Synthesize all step results into a comprehensive final output:
    
    Objective: {state['objective']}
    
    Execution Plan & Results:
    {chr(10).join([f"Step {i+1}: {state['plan'][i]}\nResult: {state['step_results'][i]}\n" 
                   for i in range(len(state['plan']))])}
    
    Reflections:
    {chr(10).join([f"{i+1}. {r}" for i, r in enumerate(state['reflections'])])}
    
    Provide a complete, coherent final output that achieves the objective."""
    
    final = llm.invoke(synthesis_prompt)
    state['final_output'] = final.content
    
    print("   ✓ Final output generated")
    return state

# Build Deep Agent graph
deep_agent_graph = StateGraph(DeepAgentState)

deep_agent_graph.add_node("plan", create_plan)
deep_agent_graph.add_node("execute", execute_step)
deep_agent_graph.add_node("reflect", reflect_on_progress)
deep_agent_graph.add_node("synthesize", synthesize_results)

deep_agent_graph.set_entry_point("plan")

deep_agent_graph.add_edge("plan", "execute")
deep_agent_graph.add_edge("execute", "reflect")

deep_agent_graph.add_conditional_edges(
    "reflect",
    should_continue_plan,
    {
        "execute": "execute",
        "synthesize": "synthesize"
    }
)

deep_agent_graph.add_edge("synthesize", END)

deep_agent = deep_agent_graph.compile()

print("✅ Deep Agent with planning & reflection created!")

In [ ]:
# Test Deep Agent
print("🧪 Testing Deep Agent")
print("="*70)

objective = """Create a comprehensive marketing campaign for BakeryAI's 
new gluten-free product line, including social media strategy, 
email campaigns, and in-store promotions."""

result = deep_agent.invoke({
    "objective": objective,
    "plan": [],
    "current_step": 0,
    "step_results": [],
    "reflections": [],
    "final_output": ""
})

print("\n" + "="*70)
print("🎯 FINAL MARKETING CAMPAIGN:")
print("="*70)
print(result['final_output'])
print("\n" + "="*70)
print(f"Steps executed: {len(result['step_results'])}")
print(f"Reflections made: {len(result['reflections'])}")

## 4. BakeryAI Deep Research Agent

Complete deep agent for research and analysis tasks.

In [ ]:
class ResearchState(TypedDict):
    research_question: str
    research_plan: List[str]
    findings: List[dict]
    synthesis: str
    quality_score: float

def research_agent_demo():
    """Demonstrate research agent capabilities"""
    
    print("\n🔬 DEEP RESEARCH AGENT DEMO")
    print("="*70)
    
    research_question = """What are the emerging trends in gluten-free baking 
    for 2025, and how should BakeryAI adapt its product line?"""
    
    print(f"\n❓ Research Question:\n{research_question}")
    
    # Simulate research process
    print("\n📋 Research Plan:")
    plan = [
        "Survey current gluten-free baking trends",
        "Analyze customer preferences and dietary requirements",
        "Review competitor offerings",
        "Identify ingredient innovations",
        "Synthesize recommendations"
    ]
    
    for i, step in enumerate(plan, 1):
        print(f"   {i}. {step}")
    
    print("\n🔍 Key Findings:")
    findings = [
        "Clean label ingredients trending (almond flour, coconut flour)",
        "Protein-enriched gluten-free products gaining popularity",
        "Customers willing to pay 15-20% premium for quality",
        "Cross-contamination concerns require dedicated facilities"
    ]
    
    for finding in findings:
        print(f"   • {finding}")
    
    print("\n💡 Recommendations:")
    recommendations = [
        "Launch premium gluten-free line with almond/coconut flour base",
        "Invest in dedicated gluten-free production space",
        "Partner with nutritionist for protein-enriched recipes",
        "Implement strict allergen testing and certification"
    ]
    
    for rec in recommendations:
        print(f"   ✓ {rec}")
    
    print("\n✅ Research Complete!")

research_agent_demo()

## 5. Key Patterns Comparison

Understanding when to use each pattern.

In [ ]:
comparison = """
┌─────────────────────────────────────────────────────────────────────┐
│                 AGENTIC AI PATTERNS COMPARISON                      │
├─────────────────┬───────────────────────────────────────────────────┤
│ Pattern         │ Best For                                          │
├─────────────────┼───────────────────────────────────────────────────┤
│ Basic           │ • Quick responses                                 │
│ Reflection      │ • Quality more important than speed               │
│                 │ • Customer-facing content                         │
│                 │ • Email, documents, reports                       │
├─────────────────┼───────────────────────────────────────────────────┤
│ Reflexion       │ • Factual accuracy critical                       │
│ with Tools      │ • Policy compliance required                      │
│                 │ • External verification needed                    │
│                 │ • Customer service, legal docs                    │
├─────────────────┼───────────────────────────────────────────────────┤
│ Deep Agent      │ • Complex, multi-step tasks                       │
│ with Planning   │ • Research and analysis                           │
│                 │ • Strategy development                            │
│                 │ • Long-running workflows                          │
├─────────────────┼───────────────────────────────────────────────────┤
│ Tree Search     │ • Multiple solution paths                         │
│ (LATS)          │ • Code generation                                 │
│                 │ • Optimization problems                           │
│                 │ • Creative ideation                               │
└─────────────────┴───────────────────────────────────────────────────┘

Performance Metrics:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Pattern           Speed    Quality   Cost      Use Case
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Basic             ★★★★★   ★★★☆☆    $         Simple tasks
Reflection        ★★★☆☆   ★★★★☆    $$        Quality content
Reflexion         ★★☆☆☆   ★★★★★    $$$       Verified facts
Deep Agent        ★☆☆☆☆   ★★★★★    $$$$      Complex research
Tree Search       ★☆☆☆☆   ★★★★★    $$$$$     Optimal solutions
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""

print(comparison)

## Summary: Deep Agents & Reflection

### ✅ What We Built:

1. **Basic Reflection Agent**: Generate → Reflect → Revise cycle
2. **Reflexion with Tools**: External verification and grounded critique
3. **Deep Agent**: Planning, sub-agents, and synthesis
4. **Research Agent**: Complete investigation workflows

### 🎯 Key Concepts:

**Reflection** = Self-critique and improvement
- Increases quality at the cost of latency
- Reduces hallucinations through verification
- Best for high-stakes outputs

**Deep Agents** = Complex task decomposition
- Break down objectives into steps
- Use specialized sub-agents
- Maintain context across long workflows
- Synthesize results into coherent output

### 💡 Production Tips:

1. **Set iteration limits** to control costs
2. **Define quality criteria** explicitly
3. **Use tools** for factual grounding
4. **Cache results** when possible
5. **Monitor performance** with LangSmith

### 📊 When to Use:

**Use Reflection when:**
- ✅ Quality > Speed
- ✅ Customer-facing content
- ✅ Errors have consequences

**Use Deep Agents when:**
- ✅ Multi-step complexity
- ✅ Research and analysis
- ✅ Long-running workflows

### 🚀 Next Steps:

1. Integrate with BakeryAI production system
2. Add human-in-the-loop for approvals
3. Implement Tree Search (LATS) for optimization
4. Deploy with LangSmith monitoring

### 📚 Resources:

- [LangChain Reflection Agents](https://blog.langchain.com/reflection-agents/)
- [Deep Agents Architecture](https://blog.langchain.com/deep-agents/)
- [Reflexion Paper](https://arxiv.org/abs/2303.11366)
- [LATS Framework](https://arxiv.org/abs/2310.04406)

**🎉 You now have advanced agentic AI capabilities for production systems!**